# 1. Setup All Workshop Dependencies

## _Note:  This Notebook Will Take A Few Minutes To Complete._

## _Please Be Patient._

In [1]:
!python --version

Python 3.11.11


In [2]:
!pip list

Package                                 Version
--------------------------------------- --------------
absl-py                                 2.1.0
accelerate                              0.34.2
adagio                                  0.2.6
aiobotocore                             2.19.0
aiohttp                                 3.9.5
aiohttp-cors                            0.7.0
aioitertools                            0.12.0
aiosignal                               1.3.2
aiosqlite                               0.19.0
alembic                                 1.14.1
altair                                  5.5.0
amazon-q-developer-jupyterlab-ext       3.4.5
amazon_sagemaker_jupyter_ai_q_developer 1.0.16
amazon_sagemaker_jupyter_scheduler      3.1.8
amazon-sagemaker-sql-editor             0.1.14
amazon-sagemaker-sql-execution          0.1.6
amazon-sagemaker-sql-magic              0.1.3
annotated-types                         0.7.0
ansi2html                               1.9.2
ansicolors      

## Package Installs

In [3]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null
!pip install --disable-pip-version-check -q awscli boto3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.36.16 which is incompatible.


In [4]:
!pip install --disable-pip-version-check -q sagemaker
!pip install --disable-pip-version-check -q smdebug
!pip install --disable-pip-version-check -q sagemaker-experiments

In [5]:
!pip install --disable-pip-version-check -q PyAthena

In [6]:
!pip install --disable-pip-version-check -q awswrangler

In [7]:
!conda install -y zip

Channels:
 - conda-forge
Platform: linux-64
Solving environment: | 

In [8]:
!pip install --disable-pip-version-check -q matplotlib

In [9]:
!pip install --disable-pip-version-check -q seaborn

In [10]:
packages_installed = True

In [11]:
%store packages_installed

Stored 'packages_installed' (bool)


In [12]:
%store

Stored variables and their in-db values:
packages_installed             -> True


# 2. Create S3 Bucket

In [20]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client("sts").get_caller_identity().get("Account")

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [14]:
setup_s3_bucket_passed = False

print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-607916531205


In [16]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

%store setup_s3_bucket_passed

{'ResponseMetadata': {'RequestId': 'E78QBVP8R8PFGHAZ', 'HostId': 'gDfkPlWNfWIBrhp9yIZfkScKaB+HQVTCwsjeCCa9bsFrGbGXuSN0saAVooFDw1uBTDMeguQNqtSSnG4U2bS8anvdKVz//tXc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'gDfkPlWNfWIBrhp9yIZfkScKaB+HQVTCwsjeCCa9bsFrGbGXuSN0saAVooFDw1uBTDMeguQNqtSSnG4U2bS8anvdKVz//tXc', 'x-amz-request-id': 'E78QBVP8R8PFGHAZ', 'date': 'Sat, 08 Feb 2025 18:42:49 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}
Stored 'setup_s3_bucket_passed' (bool)


## Verify S3 Bucket Creation

In [17]:
%store

Stored variables and their in-db values:
packages_installed                 -> True
setup_s3_bucket_passed             -> True


# 3. Copy CSV to S3 Bucket

In [19]:
%store -r packages_installed

try:
    packages_installed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Package Installation Incomplete.")
    print("+++++++++++++++++++++++++++++++")

%store -r setup_s3_bucket_passed

try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] S3 bucket not initilized")
    print("+++++++++++++++++++++++++++++++")

In [21]:
s3_csv_private_path = "s3://{}/airline-delay-cause/csv".format(bucket)
print(s3_csv_private_path)

s3://sagemaker-us-east-1-607916531205/airline-delay-cause/csv


In [23]:
!aws s3 cp "data/Airline_Delay_Cause.csv.gz" $s3_csv_private_path/
!aws s3 ls $s3_csv_private_path/

upload: data/Airline_Delay_Cause.csv.gz to s3://sagemaker-us-east-1-607916531205/airline-delay-cause/csv/Airline_Delay_Cause.csv.gz
2025-02-08 18:52:07    9970955 Airline_Delay_Cause.csv.gz


In [24]:
%store s3_csv_private_path

Stored 's3_csv_private_path' (str)


# 4. Create Athena Database Schema

In [25]:
create_base_csv_athena_db = False

%store -r s3_csv_private_path

try:
    s3_csv_private_path
except NameError:
    print("*****************************************************************************")
    print("[ERROR] CSV not copied into S3 Bucket")
    print("*****************************************************************************")

In [30]:
from pyathena import connect
import pandas as pd

database_name = "db_airline_delay_cause"

# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# Connect to Athena
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# Create a cursor object
cursor = conn.cursor()

# Execute the query
statement = f"CREATE DATABASE IF NOT EXISTS {database_name}"
cursor.execute(statement)

print(f"Database {database_name} created successfully (if it did not already exist).")


Database db_airline_delay_cause created successfully (if it did not already exist).


## Verify Successful Database Creation

In [33]:
# Execute the query
statement = "SHOW DATABASES"
cursor.execute(statement)

# Fetch results and convert to DataFrame
df_show = pd.DataFrame(cursor.fetchall(), columns=["database_name"])

# Display first 5 rows
print(df_show.head(5))

# Check if database exists
create_base_csv_athena_db = database_name in df_show["database_name"].values

%store create_base_csv_athena_db


            database_name
0  db_airline_delay_cause
1                 default
2                  dsoaws
3                  hw2_db
4  sagemaker_featurestore
Stored 'create_base_csv_athena_db' (bool)


# 5. Register S3 CSV with Athena

In [34]:
create_base_csv_athena_table = False

%store -r create_base_csv_athena_db

try:
    create_base_csv_athena_db
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Athena DB was not created")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [35]:
%store -r s3_csv_private_path
print(s3_csv_private_path)

s3://sagemaker-us-east-1-607916531205/airline-delay-cause/csv


#### Dataset columns
- `year`: YYYY format
- `month`: MM format (1-12)
- `carrier`: Code assigned by assigned by US DOT to identify a unique airline carrier.  
- `carrier_name`: Unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.
- `airport`: A three character alpha-numeric code issued by the U.S. Department of Transportation which is the official designation of the airport.
- `airport_name`: a place from which aircraft operate that usually has paved runways and maintenance facilities and often serves as a terminal
- `arr_flights`: Arrival Flights
- `arr_del15`: Arrival Delay Indicator, 15 Minutes or More Arrival delay equals the difference of the actual arrival time minus the scheduled arrival time. A flight is considered on-time when it arrives less than 15 minutes after its published arrival time.
- `carrier_ct`:	Carrier Count for airline cause of delay
- `weather_ct`:	Weather Count for airline cause of delay
- `nas_ct`:	NAS (National Air System) Count for airline cause of delay
- `security_ct`: Security County for airline cause of delay
- `late_aircraft_ct`: Late Aircraft Delay Count for airline cause of delay
- `arr_cancelled`: flight cancelled
- `arr_diverted`: flight diverted
- `arr_delay`: Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers.
- `carrier_delay`: Carrier Delay, in Minutes
- `weather_delay`: Weather Delay, in Minutes
- `nas_delay`: National Air System Delay, in Minutes
- `security_delay`: Security Delay, in Minutes
- `late_aircraft_delay`: Late Aircraft Delay, in Minutes



In [42]:
from pyathena import connect
import pandas as pd

# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# Set Athena parameters
database_name = "db_airline_delay_cause"
raw_table_name = "airline_delay_cause_csv_raw"

conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [43]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         year int,
         month int,
         carrier string,
         airport string,
         arr_flights int,
         arr_del15 int,
         carrier_ct float,
         weather_ct float,
         nas_ct float,
         security_ct float,
         late_aircraft_ct float,
         arr_cancelled int,
         arr_diverted int,
         arr_delay int,
         carrier_delay int,
         weather_delay int,
         nas_delay int,
         security_delay int,
         late_aircraft_delay int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, raw_table_name, s3_csv_private_path
)

print(statement)

pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS db_airline_delay_cause.airline_delay_cause_csv_raw(
         year int,
         month int,
         carrier string,
         airport string,
         arr_flights int,
         arr_del15 int,
         carrier_ct float,
         weather_ct float,
         nas_ct float,
         security_ct float,
         late_aircraft_ct float,
         arr_cancelled int,
         arr_diverted int,
         arr_delay int,
         carrier_delay int,
         weather_delay int,
         nas_delay int,
         security_delay int,
         late_aircraft_delay int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-607916531205/airline-delay-cause/csv'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


/tmp/ipykernel_760/731012237.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


## Verify Table Creation

In [47]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
print(df_show.head(5))

if raw_table_name in df_show.values:
    create_base_csv_athena_table = True

%store create_base_csv_athena_table

/tmp/ipykernel_760/2854214392.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


                      tab_name
0      airline_delay_cause_csv
1  airline_delay_cause_csv_raw
2             development_data
3              production_data
Stored 'create_base_csv_athena_table' (bool)


## Run a Sample Query

In [50]:
carrier = "9E"

statement = """SELECT * FROM {}.{}
    WHERE carrier = '{}' LIMIT 100""".format(
    database_name, table_name_tsv, carrier
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM db_airline_delay_cause.airline_delay_cause_csv_raw
    WHERE carrier = '9E' LIMIT 100


/tmp/ipykernel_760/1806028933.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,year,month,carrier,airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,9,9E,ABE,81,7,5.61,0.56,0.84,0.0,0.00,0,0,243,201,15,27,0,0
1,2024,9,9E,AEX,81,6,4.43,0.00,1.13,0.0,0.44,0,0,647,440,0,189,0,18
2,2024,9,9E,AGS,133,12,3.49,2.04,3.84,0.0,2.63,10,0,2382,1269,594,167,0,352
3,2024,9,9E,ALB,73,4,0.82,1.00,0.23,0.0,1.95,0,0,160,28,15,9,0,108
4,2024,9,9E,ATL,2204,280,61.70,17.36,88.36,0.0,112.57,40,4,24383,7840,2297,3690,0,10556


In [51]:
%store

Stored variables and their in-db values:
create_base_csv_athena_db                -> True
create_base_csv_athena_table             -> True
packages_installed                       -> True
s3_csv_private_path                      -> 's3://sagemaker-us-east-1-607916531205/airline-del
setup_s3_bucket_passed                   -> True


In [52]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [1]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>